In [100]:
# Met à jour pip
%pip install --upgrade pip

# Installe les datasets, torch, et torchdata
%pip install datasets==2.17.0 torch==2.2.0 torchdata==0.8.0 --quiet

# Installe les autres librairies
%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\tokenizers
      copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
      creating build\lib.win-amd64-cpython-312\tokenizers\models
      copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
      creating build\lib.win-amd64-cpython-312\tokenizers\decoders
      copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
      creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
      copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
      creating build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
      copying py_src

In [101]:
import pandas as pd
import numpy as np
import torch
import time

# Les paths de fichiers
path1 = r"C:\Users\Zakaria-Laptop\LLM-Models\NLP_TFIDF\DataBase\S08_question_answer_pairs.txt"
path2 = r"C:\Users\Zakaria-Laptop\LLM-Models\NLP_TFIDF\DataBase\S09_question_answer_pairs.txt"
path3 = r"C:\Users\Zakaria-Laptop\LLM-Models\NLP_TFIDF\DataBase\S10_question_answer_pairs.txt"

In [102]:
def read_file(path: str) -> pd.DataFrame:
    """_summary_

    Args:
        path (str): _description_

    Returns:
        pd.DataFrame: _description_
    """    
    return pd.read_csv(path, sep="	")

data1 = read_file(path1)
data2 = read_file(path2)
# It's a file encoded with ISO
data3 = pd.read_csv(path3, sep="\t", encoding='ISO-8859-1')

data = pd.concat([data1, data2, data3])
data.head(2)


,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4


## Preprocessing

In [103]:
copy_data = data.copy()
copy_data = copy_data[["Question", "Answer"]]
copy_data.head(2)

,Question,Answer
0,Was Abraham Lincoln the sixteenth President of...,yes
1,Was Abraham Lincoln the sixteenth President of...,Yes.


In [104]:
from datasets import Dataset, DatasetDict

#Diviser les données en ensembles train, validation, test
train_df = copy_data.sample(frac=0.7, random_state=42)
remaining_df = copy_data.drop(train_df.index)
validation_df = remaining_df.sample(frac=0.25, random_state=42)
test_df = remaining_df.drop(validation_df.index)

# Créer les ensembles DataSet
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

# Créer les DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset,
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 2799
    })
    validation: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 60
    })
    test: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 141
    })
})


## Le model pre-trained

In [105]:
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, AutoConfig, Trainer, TrainingArguments
import tensorflow as tf

# Load the tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name)


c:\Users\Zakaria-Laptop\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [106]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 66364418
all model parameters: 66364418
percentage of trainable model parameters: 100.00%


In [108]:
def tokenizer_function(example):
    questions = [q if isinstance(q, str) else "" for q in example["Question"]]
    answers = [a if isinstance(a, str) else "" for a in example["Answer"]]
    return tokenizer(
        questions,
        answers,
        padding= "max_length",
        truncation=True,
        return_tensors="pt"
    )
tokenized_dataset = dataset_dict.map(tokenizer_function, batched = True)
tokenized_dataset_dict = tokenized_dataset.remove_columns('__index_level_0__')
print(tokenized_dataset_dict)


Map:   0%|          | 0/2799 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', 'input_ids', 'attention_mask'],
        num_rows: 2799
    })
    validation: Dataset({
        features: ['Question', 'Answer', 'input_ids', 'attention_mask'],
        num_rows: 60
    })
    test: Dataset({
        features: ['Question', 'Answer', 'input_ids', 'attention_mask'],
        num_rows: 141
    })
})


<a name='3'></a>
## 3 - Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** fine-tuning as opposed to "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning - with comparable evaluation results as you will see soon. 

PEFT is a generic term that includes **Low-Rank Adaptation (LoRA)** and prompt tuning (which is NOT THE SAME as prompt engineering!). In most cases, when someone says PEFT, they typically mean LoRA. LoRA, at a very high level, allows the user to fine-tune their model using fewer compute resources (in some cases, a single GPU). After fine-tuning for a specific task, use case, or tenant with LoRA, the result is that the original LLM remains unchanged and a newly-trained “LoRA adapter” emerges. This LoRA adapter is much, much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

That said, at inference time, the LoRA adapter needs to be reunited and combined with its original LLM to serve the inference request.  The benefit, however, is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

In [109]:
from peft import LoraConfig, get_peft_model, TaskType

Lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_lin", "v_lin", "k_lin"],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM
)

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [110]:
# Obtenir le modèle PEFT
peft_model = get_peft_model(
    model=model,
    peft_config=Lora_config,
)

print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 884736
all model parameters: 67249154
percentage of trainable model parameters: 1.32%


<a name='3.2'></a>
### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [111]:
output_dir = f'./peft-question-answering-training-{str(int(time.time()))}'

# Arguments d'entraînement
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    num_train_epochs=1,
    learning_rate=1e-4,
    logging_steps=1,
    max_steps=1,
    )

# Initialisation du Trainer
peft_trainer = Trainer(
    model = peft_model,
    args=peft_training_args,
    train_dataset=tokenized_dataset_dict["train"],
    eval_dataset=tokenized_dataset_dict["validation"],
)

max_steps is given, it will override any value given in num_train_epochs


In [112]:
peft_trainer.train()
peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: DistilBertForQuestionAnswering.forward() got an unexpected keyword argument 'decoder_input_ids'